<a href="https://colab.research.google.com/github/devyn-miller/mgsc-mt2/blob/main/MoreGATools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
from collections import defaultdict

In [ ]:
coordinates = [[1,1], [10,1], [1,10], [5,5], [13,3], [8,2], [2,15], [3,4], [4,8], [20,20]]

In [ ]:
def random_strategy(num_cities):
  return random.sample(range(num_cities),k=num_cities)

In [ ]:
def two_point_distance(p1,p2):
  dist = ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5
  return dist

In [ ]:
miles_dict = defaultdict(lambda: None)
miles_dict[(tuple([1,1]),tuple([10,1]))] = 9

In [ ]:
for coordinate in coordinates:
  for c in coordinates:
    miles_dict[(tuple(coordinate),tuple(c))] = two_point_distance(coordinate,c)

In [ ]:
def lookup_distance(strategy):
  miles = 0
  for ix in range(len(strategy)):
    miles += miles_dict[tuple(coordinates[strategy[ix]]),tuple(coordinates[strategy[(ix+1)%len(strategy)]])]
  return miles

In [ ]:
def distance(strategy):
  miles = 0
  for index in range(len(strategy)):
    m = two_point_distance(coordinates[strategy[index]],coordinates[strategy[(index+1)%len(strategy)]])
    miles += m
  return miles

In [ ]:
def swap_mutation(strategy):
  i,j = random.sample(strategy,2)
  strategy[i],strategy[j] = strategy[j],strategy[i]
  return strategy

In [ ]:
def jumble_mutation(strategy):
  starting_point = random.randint(1,len(strategy)-2)
  window_size = random.randint(1,len(strategy[starting_point:]))
  window = strategy[starting_point:starting_point+window_size]
  jumbled_window = random.sample(window,window_size)
  new_strat = strategy[:starting_point]+jumbled_window+strategy[starting_point+window_size:]
  return new_strat

In [ ]:
def subset_mutation(strategy):
  idx1 = random.randint(0,len(strategy)-1)
  window_size = random.randint(1,len(strategy[idx1:]))
  subset = []
  base_set = []
  for i, value in enumerate(strategy):
    if i < idx1 or i > idx1 + window_size:
      base_set.append(value)
    else:
      subset.append(value)
  print(subset)
  idx2 =  random.randint(0,len(base_set))
  for i, value in enumerate(subset):
    base_set.insert(idx2+i,value)
  return base_set

In [ ]:
%%timeit
lookup_distance(list(range(10)))

4.72 µs ± 71.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
%%timeit
distance(list(range(10)))

12.1 µs ± 2.65 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
fitness_dict = defaultdict(lambda: None)

In [ ]:
def fitness_lookup(strategy):
  dist = fitness_dict[tuple(strategy)]
  if dist:
    return dist
  else:
    dist = lookup_distance(strategy)
    fitness_dict[tuple(strategy)] = dist
    return dist

In [ ]:
def elitism(population):
  fitnesses = [fitness_lookup(strat) for strat in population]
  fitnesses, sorted_population = zip(*sorted(zip(fitnesses,population)))
  elites = sorted_population[:3]
  return elites

In [ ]:
def roulette_selection_w_elitism(parent_pop,child_pop,new_pop_size):
  big_pop = parent_pop + child_pop
  new_pop = []
  new_pop += elitism(big_pop)
  for i in range(new_pop_size-3):
    sol1,sol2 = random.choices(big_pop,k=2)
    if fitness_lookup(sol1)>fitness_lookup(sol2):
      new_pop.append(sol1)
    else:
      new_pop.append(sol2)
  return new_pop

In [ ]:
def weighted_roulette(parent_pop,child_pop,new_pop_size):
  big_pop = parent_pop + child_pop
  new_pop = []
  big_pop_fit = [fitness_lookup(strategy) for strategy in big_pop]
  big_pop_prob = [(sum(big_pop_fit)- strategy_fit)/((len(big_pop)-1)*sum(big_pop_fit)) for strategy_fit in big_pop_fit]
  for i in range(new_pop_size):
    sol1,sol2 = random.choices(big_pop,weights = big_pop_prob, k=2)
    if big_pop_fit[big_pop.index(sol1)] > big_pop_fit[big_pop.index(sol2)]:
      new_pop.append(sol2)
    else:
      new_pop.append(sol1)
  return new_pop

In [ ]:
def ranked_roulette(parent_pop,child_pop,new_pop_size):
  big_pop = parent_pop + child_pop
  new_pop = []
  big_pop_fit = [fitness_lookup(strategy) for strategy in big_pop]
  sorted_big_fit, sorted_big_population = zip(*sorted(zip(big_pop_fit,big_pop),reverse=True))
  for i in range(new_pop_size):
    sol1,sol2 = random.choices(sorted_big_population,weights=list(range(1,len(big_pop)+1)),k=2)
    if big_pop_fit[big_pop.index(sol1)] > big_pop_fit[big_pop.index(sol2)]:
      new_pop.append(sol2)
    else:
      new_pop.append(sol1)
  return new_pop

In [ ]:
def knapsack_mutation(strategy,mutation_prob):
  for ix in range(len(strategy)):
    if random.random()<mutation_prob:
      strategy[ix] = 1-strategy[ix]

In [ ]:
def crossover(p1,p2):
  cross_index = random.randint(1,len(p1)-1)
  c1 = p1[:cross_index] + [point for point in p2 if point in p1[cross_index:]]
  c2 = p2[:cross_index] + [point for point in p1 if point in p1[cross_index:]]
  return c1,c2

In [ ]:
def get_children(parents,pop_size):
  children = []
  for i in range(pop_size//2):
    p1,p2=random.choices(parents,k=2)
    c1,c2=crossover(p1,p2)
    children.append(jumble_mutation(c1))
    children.append(jumble_mutation(c2))
  return children

In [ ]:
parents = [random_strategy(10) for i in range(10)]
children = [random_strategy(10) for i in range(10)]

In [ ]:
new_pop = ranked_roulette(parents,children,10)

In [ ]:
fitness_dict

defaultdict(<function __main__.<lambda>()>,
            {(0, 6, 5, 2, 3, 7, 8, 4, 1, 9): 113.98808321930319,
             (1, 8, 9, 3, 4, 6, 5, 2, 0, 7): 120.12707099756325,
             (3, 1, 7, 9, 6, 2, 4, 8, 0, 5): 104.26224934430194,
             (6, 0, 1, 8, 4, 9, 5, 2, 3, 7): 112.8836264543868,
             (6, 7, 2, 5, 4, 8, 1, 0, 3, 9): 107.16585563948956,
             (8, 3, 5, 6, 7, 1, 0, 2, 4, 9): 110.66109383391523,
             (4, 0, 2, 9, 5, 3, 7, 1, 8, 6): 111.14269998012037,
             (8, 1, 0, 6, 4, 5, 2, 9, 3, 7): 113.30648681963599,
             (7, 9, 5, 2, 1, 6, 0, 4, 8, 3): 126.35629577020086,
             (5, 0, 9, 4, 8, 7, 3, 6, 1, 2): 118.90359543150913,
             (6, 5, 4, 0, 8, 7, 3, 2, 1, 9): 104.84081089109094,
             (0, 5, 4, 3, 7, 2, 6, 8, 1, 9): 98.91656397336902,
             (7, 9, 3, 0, 2, 4, 1, 5, 6, 8): 104.67039256553443,
             (8, 9, 2, 0, 4, 3, 7, 1, 5, 6): 104.56848687883601,
             (9, 8, 0, 1, 3, 6, 7, 4, 2, 5): 120

In [ ]:
strat = [0,0,0,0,0,0,0,0,0]
knapsack_mutation(strat,0.5)
strat

[1, 1, 1, 1, 0, 1, 0, 0, 1]

In [ ]:
def traveling_ga(pop_size,num_iterations):
  population = [random_strategy(10) for i in range(pop_size)]
  for i in range(num_iterations):
    print(i)
    child_pop = get_children(population,pop_size)
    population = weighted_roulette(population,child_pop,pop_size)
  return population

In [ ]:
population = traveling_ga(100,1000)

0
1
2
3
4
5
6
7
8


ValueError: ignored